In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from pyro import clear_param_store
import pyro.contrib.gp as gp
from pyro.nn import PyroSample
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, Predictive,HMC
import torch
import kennard_stone as ks
import arviz as az
from sklearn.metrics import r2_score

In [2]:
train=pd.read_excel(r"../data/chem_train.xlsx")

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
train_no2=train.dropna(subset="no2")
X_no2=torch.tensor(train_no2.corrected_week.values).float().to(device)
y_no2=torch.tensor(train_no2.no2.values).float().to(device)

In [6]:
clear_param_store()
rbf = gp.kernels.RBF(input_dim=1)


rbf.variance = PyroSample(dist.HalfNormal(torch.tensor(train.no2.mean())))

rbf.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(10.)))

gpr = gp.models.GPRegression(X_no2,y_no2, rbf).to(device)
gpr.noise = PyroSample(dist.HalfNormal(torch.tensor(20.)))

In [7]:
nuts_kernel = NUTS(gpr.model)
mcmc = MCMC(nuts_kernel,warmup_steps=8000, num_samples=2000,num_chains=1)
mcmc.run()
torch.save(gpr, "../models/no2_seasonality_synthertic");

Sample: 100%|██████████████████████████████████████| 10000/10000 [03:52, 43.03it/s, step size=6.39e-01, acc. prob=0.926]


In [8]:
posterior_samples = mcmc.get_samples(500)
posterior_predictive= Predictive(gpr, posterior_samples)(X_no2)
prior = Predictive(gpr, num_samples=500)(X_no2)

pyro_data = az.from_pyro(mcmc,
    prior=prior,
    posterior_predictive=posterior_predictive,

)
az.to_json(pyro_data, "../Arviz_stats/mcmc_no2_seasonality_synthertic.json")

/data/michaelf/miniconda3/envs/gaussion_process/lib/python3.11/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


'../Arviz_stats/mcmc_no2_seasonality_synthertic.json'

In [9]:
train_nox=train.dropna(subset="nox")
X_nox=torch.tensor(train_nox.corrected_week.values).float().to(device)
y_nox=torch.tensor(train_nox.nox.values).float().to(device)

In [10]:
clear_param_store()
rbf_bc = gp.kernels.RBF(input_dim=1)


rbf_bc.variance = PyroSample(dist.HalfNormal(torch.tensor(train.nox.mean())))

rbf_bc.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(10.)))

gpr_bc = gp.models.GPRegression(X_nox,y_nox, rbf_bc).to(device)
gpr_bc.noise = PyroSample(dist.HalfNormal(torch.tensor(40.)))

In [11]:
nuts_kernel_bc = NUTS(gpr_bc.model)


mcmc_bc = MCMC(nuts_kernel_bc,warmup_steps=8000, num_samples=2000,num_chains=1)

mcmc_bc.run()
torch.save(gpr_bc, "../models/nox_seasonality_synthertic");

Sample: 100%|██████████████████████████████████████| 10000/10000 [03:08, 53.12it/s, step size=9.16e-01, acc. prob=0.859]


In [12]:
posterior_samples_bc = mcmc_bc.get_samples(500)
posterior_predictive_bc = Predictive(gpr_bc, posterior_samples_bc)(X_nox)
prior_bc = Predictive(gpr_bc, num_samples=500)(X_nox)

pyro_data_bc = az.from_pyro(mcmc_bc,
    prior=prior_bc,
    posterior_predictive=posterior_predictive_bc,

)
az.to_json(pyro_data_bc, "../Arviz_stats/mcmc_nox_seasonality_synthertic.json")

/data/michaelf/miniconda3/envs/gaussion_process/lib/python3.11/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


'../Arviz_stats/mcmc_nox_seasonality_synthertic.json'